In [2]:
'''
---------------------------------------------------------------
Simulation for Function-on-Function Linear Regression
---------------------------------------------------------------
* 1 d

* 1 Response Y & 3 Predictor (X1, X2, X3)   

* Regular Dense Grid (num of grid: 21)

* Training samp size: 200
  Test     samp size: 100

* Using Epan Kernal Function in LPR

* Using Leave-One-Curve-Out CV for bandwidth selection in LPR 
---------------------------------------------------------------
'''

#####
from os import chdir
chdir('D:/Work_Jupyter/敏')
import FFReg
import Plotting
import numpy as np
from collections import namedtuple
import time

chdir('D:/Work_Jupyter/敏/SIM_1y3x_grid')
###

sim_rep_time = 100

tran_samp_size = 200
test_samp_size = 100

num_pts = 21

num_grid_X1 = 21
num_grid_X2 = 21
num_grid_X3 = 21
num_grid_Y  = 21


In [4]:
time_grid_X1 = np.linspace(0, 5, 21)
time_grid_X2 = np.linspace(0, 10, 21)
time_grid_X3 = np.linspace(0, 10, 21)
time_grid_Y = np.linspace(0, 5, 21)


In [5]:
grid_width = time_grid_Y[1] - time_grid_Y[0]
grid_width

0.25

In [6]:

# bw for X1
X1_candidate_h_mean = np.arange(0.3, 0.6, 0.05).reshape(-1, 1)
X1_candidate_h_cov = np.arange(0.3, 0.6, 0.05).repeat(2).reshape(-1, 2)
X1_candidate_h_diag_cov = np.arange(0.3, 0.8, 0.05).reshape(-1, 1)

X2_candidate_h_mean = np.arange(0.6, 1, 0.05).reshape(-1, 1)
X2_candidate_h_cov = np.arange(0.6, 1, 0.05).repeat(2).reshape(-1, 2)
X2_candidate_h_diag_cov =  np.arange(0.6, 1, 0.05).reshape(-1, 1)

X3_candidate_h_mean = np.arange(0.6, 1, 0.05).reshape(-1, 1)
X3_candidate_h_cov = np.arange(0.6, 1, 0.05).repeat(2).reshape(-1, 2)
X3_candidate_h_diag_cov =  np.arange(0.6, 1, 0.05).reshape(-1, 1)

Y_candidate_h_mean = np.arange(0.3, 0.8, 0.05).reshape(-1, 1)
Y_candidate_h_cov = np.arange(0.3, 0.8, 0.05).repeat(2).reshape(-1, 2)


# bandwidth
X1_Y_candidate_h_cov = np.asanyarray(np.meshgrid(np.linspace(0.3, 0.7, 5), 
                                                 np.linspace(0.3, 0.7, 5))).T.reshape(-1,2)
X2_Y_candidate_h_cov = np.asanyarray(np.meshgrid(np.linspace(0.4, 0.7, 4), 
                                                 np.linspace(0.26, 0.36, 3))).T.reshape(-1,2)
X3_Y_candidate_h_cov = np.asanyarray(np.meshgrid(np.linspace(0.6, 1, 5), 
                                                np.linspace(0.3, 0.55, 6))).T.reshape(-1,2)
X1_X2_candidate_h_cov = np.asanyarray(np.meshgrid(np.linspace(0.4, 0.6, 3), 
                                                np.linspace(0.65, 0.85, 3))).T.reshape(-1,2)
X1_X3_candidate_h_cov = np.asanyarray(np.meshgrid(np.linspace(0.3, 0.7, 5), 
                                                np.linspace(0.7, 0.9, 3))).T.reshape(-1,2)
X2_X3_candidate_h_cov = np.asanyarray(np.meshgrid(np.linspace(0.6, 1, 5), 
                                                np.linspace(0.6, 1, 5))).T.reshape(-1,2)

In [7]:
name = ['X1', 'X2', 'X3', 'Y']

In [ ]:
np.random.seed(104)
my_MISE   = np.zeros(sim_rep_time)
for i in range(sim_rep_time):
    
    #load simulation data

    datas = []
    datas.append(namedtuple('X1_data', ['obs_tran', 'obs_test', 'real_val_on_grid', 'tran_time_pts', 'test_time_pts', 'time_grid'], 
                             verbose=False))
    datas.append(namedtuple('X2_data', ['obs_tran', 'obs_test', 'real_val_on_grid', 'tran_time_pts', 'test_time_pts', 'time_grid'], 
                             verbose=False))
    datas.append(namedtuple('X3_data', ['obs_tran', 'obs_test', 'real_val_on_grid', 'tran_time_pts', 'test_time_pts', 'time_grid'], 
                             verbose=False))
    datas.append(namedtuple('Y_data', ['obs_tran', 'obs_test', 'real_val_on_grid', 'tran_time_pts', 'test_time_pts', 'time_grid'], 
                             verbose=False))

    for j in range(len(datas)):
        datas[j].obs_tran = np.loadtxt('tran '+ name[j] + ' of samp ' + np.str(i + 1),delimiter = ' ').T
        datas[j].obs_test = np.loadtxt('test '+ name[j] + ' of samp ' + np.str(i + 1), delimiter = ' ').T
        datas[j].real_val_on_grid = np.loadtxt('real_on_grid '+ name[j] + ' of samp ' + np.str(i + 1), delimiter = ' ').T
        datas[j].tran_time_pts = np.loadtxt('tran_time_pt '+ name[j] + ' of samp ' + np.str(i + 1), delimiter = ' ').T
        datas[j].test_time_pts = np.loadtxt('test_time_pt '+ name[j] + ' of samp ' + np.str(i + 1), delimiter = ' ').T
    X1_data, X2_data, X3_data, Y_data = datas
    
    X1_data.time_grid, X2_data.time_grid, X3_data.time_grid = time_grid_X1, time_grid_X2, time_grid_X3
    Y_data.time_grid = time_grid_Y
    '''
    Results of X1
    '''

    #------------------------
    # Get FPCA result of X1
    #------------------------
    start_time = time.time()
    result_X1 = FFReg.Get_FPCA_Result(X1_data, X1_candidate_h_mean, 
                                      X1_candidate_h_cov, X1_candidate_h_diag_cov, 
                                      ker_fun = 'Epan')


    #fitting test sample
    fit_test_X1 =result_X1.Restruect_Fun(X1_data.test_time_pts.reshape(test_samp_size, num_pts, 1),
                                         X1_data.obs_test)[1]

    '''
    Results of X2
    '''


    #------------------------
    # Get FPCA result of X2
    #------------------------
    result_X2 = FFReg.Get_FPCA_Result(X2_data, X2_candidate_h_mean, 
                                      X2_candidate_h_cov, X2_candidate_h_diag_cov,
                                      ker_fun = 'Epan')



    fit_test_X2 =result_X2.Restruect_Fun(X2_data.test_time_pts.reshape(test_samp_size, num_pts, 1),
                                         X2_data.obs_test)[1]

    '''
    Results of X3
    '''

    #------------------------
    # Get FPCA result of X3
    #------------------------

    result_X3 = FFReg.Get_FPCA_Result(X3_data, X3_candidate_h_mean, 
                                      X3_candidate_h_cov, X3_candidate_h_diag_cov, ker_fun = 'Epan')
    


    fit_test_X3 =result_X3.Restruect_Fun(X3_data.test_time_pts.reshape(test_samp_size, num_pts, 1),
                                         X3_data.obs_test)[1]


    '''
    Results of Y
    '''

    #------------------------
    # Get FPCA result of Y
    #------------------------
    result_Y = FFReg.Fit_Mean_and_Cov(Y_data, Y_candidate_h_mean, 
                                      Y_candidate_h_cov, ker_fun = 'Epan')


    '''
    Construct Coefficient Function

        1. fit cov_Xi_Xj
        2. fit cov_Xi_Y
        3. combine them to build coefficient function

    '''


    fit_cov_X1_Y = FFReg.Fit_Cov_XY(X_time_pts = X1_data.tran_time_pts,
                                    Y_time_pts = Y_data.tran_time_pts,
                                    obs_X = X1_data.obs_tran, 
                                    obs_Y = Y_data.obs_tran,
                                    X_time_grid = X1_data.time_grid,
                                    Y_time_grid = Y_data.time_grid, 
                                    fit_X_mean = result_X1.mean_fun, 
                                    fit_Y_mean = result_Y.mean_fun, 
                                    candidate_h_cov = X1_Y_candidate_h_cov, 
                                    ker_fun = 'Epan')



    # fit Cov
    fit_cov_X2_Y = FFReg.Fit_Cov_XY(X_time_pts = X2_data.tran_time_pts,
                                    Y_time_pts = Y_data.tran_time_pts,
                                    obs_X = X2_data.obs_tran, 
                                    obs_Y = Y_data.obs_tran,
                                    X_time_grid = X2_data.time_grid,
                                    Y_time_grid = Y_data.time_grid,
                                    fit_X_mean = result_X2.mean_fun,
                                    fit_Y_mean = result_Y.mean_fun,
                                    candidate_h_cov = X2_Y_candidate_h_cov, 
                                    ker_fun = 'Epan')


    fit_cov_X3_Y = FFReg.Fit_Cov_XY(X_time_pts = X3_data.tran_time_pts,
                                    Y_time_pts = Y_data.tran_time_pts,
                                    obs_X = X3_data.obs_tran, 
                                    obs_Y = Y_data.obs_tran,
                                    X_time_grid = X3_data.time_grid,
                                    Y_time_grid = Y_data.time_grid,
                                    fit_X_mean = result_X3.mean_fun,
                                    fit_Y_mean = result_Y.mean_fun,
                                    candidate_h_cov = X3_Y_candidate_h_cov, 
                                    ker_fun = 'Epan')


    fit_cov_X1_X2 = FFReg.Fit_Cov_XY(X_time_pts = X1_data.tran_time_pts,
                                     Y_time_pts = X2_data.tran_time_pts,
                                     obs_X = X1_data.obs_tran,
                                     obs_Y = X2_data.obs_tran,
                                     X_time_grid = X1_data.time_grid,
                                     Y_time_grid = X2_data.time_grid,
                                     fit_X_mean = result_X1.mean_fun,
                                     fit_Y_mean = result_X2.mean_fun,
                                     candidate_h_cov = X1_X2_candidate_h_cov, 
                                     ker_fun = 'Epan')





    fit_cov_X1_X3 = FFReg.Fit_Cov_XY(X_time_pts = X1_data.tran_time_pts,
                                     Y_time_pts = X3_data.tran_time_pts,
                                     obs_X = X1_data.obs_tran,
                                     obs_Y = X3_data.obs_tran,
                                     X_time_grid = X1_data.time_grid,
                                     Y_time_grid = X3_data.time_grid,
                                     fit_X_mean = result_X1.mean_fun, 
                                     fit_Y_mean = result_X3.mean_fun, 
                                     candidate_h_cov = X1_X3_candidate_h_cov, 
                                     ker_fun = 'Epan')



    fit_cov_X2_X3 = FFReg.Fit_Cov_XY(X_time_pts = X2_data.tran_time_pts,
                                     Y_time_pts = X3_data.tran_time_pts,
                                     obs_X = X2_data.obs_tran, 
                                     obs_Y = X3_data.obs_tran,
                                     X_time_grid = X2_data.time_grid,
                                     Y_time_grid = X3_data.time_grid, 
                                     fit_X_mean = result_X2.mean_fun, 
                                     fit_Y_mean = result_X3.mean_fun, 
                                     candidate_h_cov = X2_X3_candidate_h_cov, 
                                     ker_fun = 'Epan')




    '''
    Coefficient Function
    '''
    #==========================================
    # Get Block_cov_XX (combine all cov_Xi_Xj)
    #==========================================


    block_cov_XX = np.bmat([[result_X1.cov_fun, fit_cov_X1_X2, fit_cov_X1_X3],
                            [fit_cov_X1_X2.T, result_X2.cov_fun, fit_cov_X2_X3],
                            [fit_cov_X1_X3.T, fit_cov_X2_X3.T, result_X3.cov_fun]])


    
    inv_block_cov_XX = np.linalg.pinv(block_cov_XX, rcond= 0.05)



    #==========================================
    # Get Block_cov_XY (combine all cov_Xi_Yj)
    #==========================================

    block_cov_XY = np.bmat([[fit_cov_X1_Y], [fit_cov_X2_Y], [fit_cov_X3_Y]])


    
    # -----------------------------
    # Compute Coefficient Function
    # -----------------------------

    fit_Beta_without_delta =  np.matmul(inv_block_cov_XX,block_cov_XY)

    ## add delta
    delta = np.diag(np.repeat([X1_data.time_grid[1] - X1_data.time_grid[0],
                               X2_data.time_grid[1] - X2_data.time_grid[0],
                               X3_data.time_grid[1] - X3_data.time_grid[0]],
                               [num_grid_X1, num_grid_X2, num_grid_X3]))

    delta_inv = np.linalg.inv(delta)
    fit_Beta  = np.matmul(delta_inv, fit_Beta_without_delta)

    '''
    1. Fit the Response on grid pts (for both training Y and test Y)
    2. Plot the Result
    '''



    #---------------------------------------
    # fit Response of test set on grid
    #---------------------------------------
    fit_test_X_center = np.bmat([fit_test_X1 - result_X1.mean_fun,
                                 fit_test_X2 - result_X2.mean_fun,
                                 fit_test_X3 - result_X3.mean_fun])

    #compute test_Y(on grid pts)
    fit_test_Y = result_Y.mean_fun + np.matmul(fit_Beta_without_delta.T, fit_test_X_center.T).T

    
    #real value of test_Y (on grid pts)
    real_test_Y = Y_data.real_val_on_grid[tran_samp_size:,:]

   
    #======================================
    #compute Mean integrated squared error
    #======================================
    

    my_MISE[i] = np.sum(np.square(real_test_Y - fit_test_Y) * grid_width, 1).mean()
    print(i, ", time = ", time.time() - start_time)
    np.savetxt('MISE_1y3x_grid', my_MISE, newline='\n', delimiter = ' ', fmt="%s")


Bandwidth of cov:  [ 0.7  0.7]
Bandwidth of cov:  [ 0.4   0.26]
Bandwidth of cov:  [ 0.6   0.55]
Bandwidth of cov:  [ 0.4   0.65]
Bandwidth of cov:  [ 0.3  0.7]
Bandwidth of cov:  [ 0.6  0.6]
0 , time =  38.635804891586304
Bandwidth of cov:  [ 0.7  0.7]
Bandwidth of cov:  [ 0.7   0.36]
Bandwidth of cov:  [ 1.    0.55]
Bandwidth of cov:  [ 0.4   0.65]
Bandwidth of cov:  [ 0.3  0.7]
Bandwidth of cov:  [ 0.6  0.6]
1 , time =  36.73522710800171
Bandwidth of cov:  [ 0.3  0.3]
Bandwidth of cov:  [ 0.4   0.26]
Bandwidth of cov:  [ 0.6  0.3]
Bandwidth of cov:  [ 0.6   0.85]
Bandwidth of cov:  [ 0.7  0.7]
Bandwidth of cov:  [ 0.6  0.6]
2 , time =  36.639939308166504
Bandwidth of cov:  [ 0.3  0.3]
Bandwidth of cov:  [ 0.4   0.26]
Bandwidth of cov:  [ 0.6   0.55]
Bandwidth of cov:  [ 0.6   0.75]
Bandwidth of cov:  [ 0.3  0.7]
Bandwidth of cov:  [ 0.6  0.6]
3 , time =  36.51195430755615
Bandwidth of cov:  [ 0.7  0.7]
Bandwidth of cov:  [ 0.5   0.36]
Bandwidth of cov:  [ 1.    0.55]
Bandwidth of co

Bandwidth of cov:  [ 0.7  0.7]
Bandwidth of cov:  [ 0.7   0.36]
Bandwidth of cov:  [ 0.8   0.55]
Bandwidth of cov:  [ 0.6   0.65]
Bandwidth of cov:  [ 0.3  0.7]
Bandwidth of cov:  [ 0.6  0.6]
40 , time =  36.225852251052856
Bandwidth of cov:  [ 0.7  0.7]
Bandwidth of cov:  [ 0.6   0.26]
Bandwidth of cov:  [ 1.    0.55]
Bandwidth of cov:  [ 0.6   0.85]
Bandwidth of cov:  [ 0.3  0.7]
Bandwidth of cov:  [ 0.6  0.6]
41 , time =  36.229434967041016
Bandwidth of cov:  [ 0.3  0.3]
Bandwidth of cov:  [ 0.6   0.26]
Bandwidth of cov:  [ 0.6   0.55]
Bandwidth of cov:  [ 0.6   0.65]
Bandwidth of cov:  [ 0.7  0.8]
Bandwidth of cov:  [ 0.6  0.6]
42 , time =  36.11291432380676
Bandwidth of cov:  [ 0.7  0.7]
Bandwidth of cov:  [ 0.6   0.26]
Bandwidth of cov:  [ 1.    0.55]
Bandwidth of cov:  [ 0.6   0.85]
Bandwidth of cov:  [ 0.3  0.7]
Bandwidth of cov:  [ 0.6  0.6]
43 , time =  36.285794258117676
Bandwidth of cov:  [ 0.7  0.7]
Bandwidth of cov:  [ 0.4   0.26]
Bandwidth of cov:  [ 0.7   0.55]
Bandwidt

Bandwidth of cov:  [ 0.7  0.9]
Bandwidth of cov:  [ 0.6  0.6]
79 , time =  36.3587281703949
Bandwidth of cov:  [ 0.7  0.3]
Bandwidth of cov:  [ 0.7   0.36]
Bandwidth of cov:  [ 0.6  0.3]
Bandwidth of cov:  [ 0.4   0.65]
Bandwidth of cov:  [ 0.7  0.7]
Bandwidth of cov:  [ 0.6  0.6]
80 , time =  36.44426894187927
Bandwidth of cov:  [ 0.7  0.7]
Bandwidth of cov:  [ 0.4   0.36]
Bandwidth of cov:  [ 0.6  0.3]
Bandwidth of cov:  [ 0.4   0.65]
Bandwidth of cov:  [ 0.3  0.7]
Bandwidth of cov:  [ 0.6  0.6]
81 , time =  36.65445280075073
Bandwidth of cov:  [ 0.3  0.3]
Bandwidth of cov:  [ 0.6   0.31]
Bandwidth of cov:  [ 1.    0.55]
Bandwidth of cov:  [ 0.4   0.65]
Bandwidth of cov:  [ 0.7  0.7]
Bandwidth of cov:  [ 0.6  0.6]
82 , time =  36.439674377441406
Bandwidth of cov:  [ 0.7  0.7]
Bandwidth of cov:  [ 0.7   0.36]
Bandwidth of cov:  [ 1.    0.55]
Bandwidth of cov:  [ 0.6   0.65]
Bandwidth of cov:  [ 0.7  0.7]
Bandwidth of cov:  [ 0.6  0.6]
83 , time =  36.458725929260254
Bandwidth of cov: 

In [3]:
my_MISE = np.loadtxt('MISE_1y3x_grid')

In [4]:
refund_ff_MISE = np.loadtxt('refundFF_MISE', delimiter = ' ')
refund_fpc_MISE = np.loadtxt('refundFPC_MISE', delimiter = ' ')

In [5]:
# create the quantile value of each array
# save in tuple

quantile_my_MISE  = []
quantile_refund_ff_MISE  = []
quantile_refund_fpc_MISE  = []

#add mean        
quantile_my_MISE.append(np.asscalar(np.mean(my_MISE)))
quantile_refund_ff_MISE.append(np.asscalar(np.mean(refund_ff_MISE)))
quantile_refund_fpc_MISE.append(np.asscalar(np.mean(refund_fpc_MISE)))

#add median
quantile_my_MISE.append(np.asscalar(np.median(my_MISE)))
quantile_refund_ff_MISE.append(np.asscalar(np.median(refund_ff_MISE)))
quantile_refund_fpc_MISE.append(np.asscalar(np.median(refund_fpc_MISE)))

#add std
quantile_my_MISE.append(np.asscalar(np.std(my_MISE)))
quantile_refund_ff_MISE.append(np.asscalar(np.std(refund_ff_MISE)))
quantile_refund_fpc_MISE.append(np.asscalar(np.std(refund_fpc_MISE)))

#convert to tuple
quantile_my_MISE = tuple(quantile_my_MISE)
quantile_refund_ff_MISE = tuple(quantile_refund_ff_MISE)
quantile_refund_fpc_MISE = tuple(quantile_refund_fpc_MISE)


In [6]:
# print table
print("====================================================")
print("      Compare Mean Integrated Squared Error         ")
print("====================================================")
print("|   n = 200    |   mean   |   median   |    std    |")
print("----------------------------------------------------")
print("   my  method  " ,3 * " %10.3f"%quantile_my_MISE[:3])
print("----------------------------------------------------")
print("   refund_ff   " ,3 * " %10.3f"%quantile_refund_ff_MISE[:3])
print("----------------------------------------------------")
print("   refund_fpc  " ,3 * " %10.3f"%quantile_refund_fpc_MISE[:3])
print("====================================================")


      Compare Mean Integrated Squared Error         
|   n = 200    |   mean   |   median   |    std    |
----------------------------------------------------
   my  method         2.231      2.212      0.314
----------------------------------------------------
   refund_ff          2.290      2.293      0.257
----------------------------------------------------
   refund_fpc         1.636      1.619      0.209
